In [104]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math

In [105]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

#load play tennis dataset
play_tennis =  pd.read_csv('play_tennis.csv')
play_tennis = play_tennis.drop('day',axis=1)

In [106]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [107]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [108]:
iris_y.rename(columns = {0:4}, inplace = True) 

In [109]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [110]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [111]:
iris.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3],4:"target"}, inplace = True)

In [112]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [113]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [114]:
def best_attribute(data,target_attribute):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if information_gain(data, i, target_attribute) > gain_attribute['value']:
                gain_attribute['value'] = information_gain(data, i, target_attribute)
                gain_attribute['name'] = i

    return gain_attribute['name']

In [115]:
entropy(play_tennis,"play")

0.9402859586706309

In [116]:
information_gain(play_tennis, "outlook", "play")

0.2467498197744391

In [117]:
best_attribute(play_tennis,'play')

'outlook'

In [239]:
import math

class Node:
    def __init__(self, attribute=None, label=None):
        self.attribute = attribute
        self.label = label
        self.children = {}
        self.isDaun = False
  
    def setIsDaun(self, isDaun):
        self.isDaun = isDaun
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def isEmpty(self):
        return len(self.children)==0
    
    def getLabel(self):
        return self.label

In [240]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [241]:
play_tennis['play'].unique()[0]

'No'

In [290]:
def id3(data, target_attribute):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(play_tennis.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute))
            
    return node

In [291]:
play_tennis['play'][0]

'No'

In [292]:
id3(play_tennis,"play")

In [345]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +node.label)
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +i)
            print_tree(node.children[i],depth+1)        
        

In [346]:
print_tree(id3(play_tennis, "play"),0)

[outlook]
----Sunny
    [humidity]
--------High
            No
--------Normal
            Yes
----Overcast
        Yes
----Rain
    [wind]
--------Weak
            Yes
--------Strong
            No


In [319]:
print(id3(play_tennis, "play").children['Sunny'].children['High'].label)

No
